In [1]:
import sys
sys.path.append('../')
import trimesh
import networkx as nx
import torch
from models.gan_model import *
from models.networks import *

from tqdm import tqdm
from torch.utils.data import Dataset
from pathlib import Path

In [2]:
device = 'cuda'
NUM_VERTEX = 40
z_size = 100
epochs = 100
adj_gan = GraphGAN(num_vertex=NUM_VERTEX, lr=1e-4, device=device)
feature_gan = GraphGAN(num_vertex=NUM_VERTEX, generator=FeatureGenerator(z_size, NUM_VERTEX, 3), discriminator=FeatureDiscriminator(NUM_VERTEX, 3), lr=1e-4, device=device)

In [53]:
adj_gan.load_state_dict(torch.load(f'../models/weigths/adj_gan_{epochs}_{NUM_VERTEX}'))

<All keys matched successfully>

In [54]:
feature_gan.load_state_dict(torch.load(f'../models/weigths/feature_gan_{epochs}_{NUM_VERTEX}'))

<All keys matched successfully>

In [3]:
graph_output = adj_gan.generate_adj(return_symmetric=True)
graph_output = graph_output.cpu().detach().numpy()

In [4]:
feature_matrix = feature_gan.generate_feat().cpu().detach().numpy().squeeze(0)

In [5]:
triangles = []
for i in range(len(graph_output)):
   for j in range(i+1, len(graph_output)):
       if graph_output[i, j] == 1:
           for k in range(j+1, len(graph_output)):
               if graph_output[j, k] == 1 and graph_output[k, i] == 1:
                  triangles.append([i, j, k])

In [62]:
import trimesh
import numpy as np

# create faces
faces = np.array(triangles)

# create a trimesh
mesh = trimesh.Trimesh(vertices=feature_matrix, faces=faces)
mesh.export('40vertex-sheet-100epochs_2.stl');